In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [18]:
base_model = EfficientNetB0(weights='imagenet', include_top= False, input_shape=(224, 224, 3))


In [19]:
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_6 (Rescaling)        (None, 224, 224, 3)  0           ['input_4[0][0]']                
                                                                                                  
 normalization_3 (Normalization  (None, 224, 224, 3)  7          ['rescaling_6[0][0]']            
 )                                                                                                
                                                                                     

                                                                                                  
 block2a_project_conv (Conv2D)  (None, 56, 56, 24)   2304        ['block2a_se_excite[0][0]']      
                                                                                                  
 block2a_project_bn (BatchNorma  (None, 56, 56, 24)  96          ['block2a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 56, 56, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 56, 56, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 block3b_expand_bn (BatchNormal  (None, 28, 28, 240)  960        ['block3b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3b_expand_activation (Act  (None, 28, 28, 240)  0          ['block3b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 28, 28, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_b

 block4b_dwconv (DepthwiseConv2  (None, 14, 14, 480)  4320       ['block4b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block4b_bn (BatchNormalization  (None, 14, 14, 480)  1920       ['block4b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block4b_activation (Activation  (None, 14, 14, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePoolin

                                                                                                  
 block5a_activation (Activation  (None, 14, 14, 480)  0          ['block5a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
          

                                                                                                  
 block5c_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5c_se_reduce[0][0]']      
                                                                                                  
 block5c_se_excite (Multiply)   (None, 14, 14, 672)  0           ['block5c_activation[0][0]',     
                                                                  'block5c_se_expand[0][0]']      
                                                                                                  
 block5c_project_conv (Conv2D)  (None, 14, 14, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 14, 14, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
          

                                                                                                  
 block6b_project_bn (BatchNorma  (None, 7, 7, 192)   768         ['block6b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6b_drop (Dropout)         (None, 7, 7, 192)    0           ['block6b_project_bn[0][0]']     
                                                                                                  
 block6b_add (Add)              (None, 7, 7, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6b_add[0][0]']            
          

                                                                                                  
 block6d_add (Add)              (None, 7, 7, 192)    0           ['block6d_drop[0][0]',           
                                                                  'block6c_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6d_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 7, 7, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation) 

In [20]:
num_layers = len(base_model.layers)
print(num_layers)
freeze_layers = (num_layers - 10)  # Freeze 1/4 of the layers
print(freeze_layers)
for layer in base_model.layers[:freeze_layers]:
    layer.trainable = False

for layer in base_model.layers[freeze_layers:]:
    layer.trainable = True


238
228


In [40]:
from tensorflow.keras.layers import BatchNormalization
model = models.Sequential()
model.add(base_model)
model.add(layers.Dense(128, activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))  # Use the number of classes you have


In [41]:
model.compile(optimizer=optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])


In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rescale the images and apply data augmentation for the training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

# Only rescale the images for the validation and testing sets
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators for the training, validation, and testing sets
train_generator = train_datagen.flow_from_directory('C:/Users/manug/Big data/data/archive/New_data/real_and_fake_face/',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory('C:/Users/manug/Big data/data/archive/New_data/real_and_fake_face/',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='categorical')

test_generator = test_datagen.flow_from_directory('C:/Users/manug/Big data/data/archive/New_data/real_and_fake_face/',
                                                  target_size=(64, 64),
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  shuffle=False)


Found 2041 images belonging to 2 classes.
Found 2041 images belonging to 2 classes.
Found 2041 images belonging to 2 classes.


In [43]:
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=30,
                    validation_data=val_generator,
                    validation_steps=len(val_generator))


Epoch 1/30
64/64 [==============================] - 146s 2s/step - loss: 0.7206 - accuracy: 0.4953 - val_loss: 0.6941 - val_accuracy: 0.4704
Epoch 2/30
64/64 [==============================] - 90s 1s/step - loss: 0.6979 - accuracy: 0.5110 - val_loss: 0.7032 - val_accuracy: 0.5296
Epoch 3/30
64/64 [==============================] - 230s 4s/step - loss: 0.6998 - accuracy: 0.4904 - val_loss: 0.6922 - val_accuracy: 0.5296
Epoch 4/30
64/64 [==============================] - 1901s 30s/step - loss: 0.7022 - accuracy: 0.5012 - val_loss: 0.6918 - val_accuracy: 0.5296
Epoch 5/30
64/64 [==============================] - 179s 3s/step - loss: 0.6949 - accuracy: 0.5120 - val_loss: 0.7063 - val_accuracy: 0.5296
Epoch 6/30
64/64 [==============================] - 193s 3s/step - loss: 0.6952 - accuracy: 0.5223 - val_loss: 0.6914 - val_accuracy: 0.5296
Epoch 7/30
64/64 [==============================] - 203s 3s/step - loss: 0.6946 - accuracy: 0.5169 - val_loss: 0.6924 - val_accuracy: 0.5296
Epoch 8/30
6

In [34]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))

# Make predictions
predictions = model.predict(test_generator)


24/24 [==============================] - 24s 872ms/step


In [9]:
predictions

array([[0.5277071 , 0.47229293],
       [0.49063113, 0.5093689 ],
       [0.48700425, 0.5129957 ],
       [0.52272105, 0.4772789 ],
       [0.5277085 , 0.47229153],
       [0.49610835, 0.50389165],
       [0.31080425, 0.68919575],
       [0.38925382, 0.61074615],
       [0.5815544 , 0.41844568],
       [0.51377267, 0.48622727],
       [0.4515158 , 0.5484842 ],
       [0.5031407 , 0.4968593 ],
       [0.4988283 , 0.5011717 ],
       [0.44573796, 0.55426204],
       [0.5024518 , 0.49754828],
       [0.5895331 , 0.41046697],
       [0.45514148, 0.5448585 ],
       [0.4154822 , 0.58451784],
       [0.52140963, 0.47859043],
       [0.51549524, 0.4845047 ],
       [0.47208384, 0.5279161 ],
       [0.46966034, 0.53033966],
       [0.3651186 , 0.6348814 ],
       [0.49728012, 0.5027199 ],
       [0.44685343, 0.5531466 ],
       [0.48378325, 0.5162168 ],
       [0.5019487 , 0.49805123],
       [0.49916953, 0.5008305 ],
       [0.3308551 , 0.6691449 ],
       [0.50453573, 0.49546432],
       [0.